<a href="https://colab.research.google.com/github/Syeda-Shamama/RAG_project/blob/main/RAG_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Retrieval-Augmented Generation (RAG) for AI-Powered Question Answering**

In this project, we are building an AI-powered question-answering system using Retrieval-Augmented Generation (RAG), a technique that enhances AI responses by combining retrieval and generation. Instead of relying solely on a language model’s internal knowledge, our system first retrieves relevant documents from an external source (a vector database) and then uses that information to generate accurate, context-aware answers.



##  **Install Required Libraries**

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

## **Import Required Libraries**

In [ ]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

## **Set Up API keys**

In [ ]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
cp = Pinecone(api_key=pinecone_api_key)

import os
os.environ ['GOOGLE_API_KEY'] = userdata.get ('GOOGLE_API_KEY')

## **Set Up Pinecone Index**
python
Copy
Edit


In [ ]:
import time
index_name = "pro"# change if desired
cp.create_index(
name=index_name,
dimension=768,
metric="cosine",
spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
time.sleep(2)
index = cp.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")  # Change model name


## **Convert Text into Vectors**

In [ ]:
vector = embeddings.embed_query("hello, world!")

In [ ]:
vector[:3]

[0.05168594419956207, -0.030764883384108543, -0.03062233328819275]

## **Set Up Pinecone Vector Store**

In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
# Data save

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

## **Store Documents in Pinecone**

In [ ]:
# Adding to vectorstore
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['5154cf6f-8b2a-4d04-8a37-8dc151f7e7ca',
 '156563e9-032c-47cf-9aa1-59829dbc4dd4',
 '68798b51-4a61-40ac-a3d5-370b48b7df1e',
 '24641ae1-a9e6-4b9d-bc4b-3f0846280bea',
 '43ea8830-cffa-4d34-98d5-c6acf48c637f',
 'fe4fb3c3-63af-400f-974c-1b800624e3ab',
 'f59ee4b5-938d-413e-b6f1-087340c9dd3e',
 'b990043e-8508-42bf-b423-1f6beb289746',
 '23059d4b-7d53-42f1-b6b8-d379183409d8',
 'e8a9acf0-08ca-4b10-8d2f-f3c7dc8716e9']

## **Search for Similar Documents**

In [ ]:
# Data Reterive
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "There are many things happening in the country, nowdays",
    k=3,
    filter={"source": "news"},
)
for res,score in results:
    print(f"* SIM={score:3f}] {res.page_content} [{res.metadata}]")

* SIM=0.570734] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* SIM=0.521469] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* SIM=0.500749] Robbers broke into the city bank and stole $1 million in cash. [{'source': 'news'}]


## **Initialize Google Gemini LLM**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## **Define Function to Answer User Queries**

In [ ]:
def answer_to_user(query: str):

    if "vector_store" not in globals():
        raise ValueError("Error: vector_store is not defined. Initialize it before calling this function.")

    results = vector_store.similarity_search(query, k=3)

    print(f"Found {len(results)} results.")

    context_texts = [doc.page_content for doc in results]
    context_str = "\n".join(context_texts)

    prompt = f"Answer the user's query: {query}. Use these references to help: {context_str}
    final_answer = llm.invoke(prompt)

    return final_answer

query = "What is breakfast?"
response = answer_to_user(query)
print(response)
print(response.content)


Found 3 results.
content='Based on the provided text, breakfast is a meal that can include chocolate chip pancakes and scrambled eggs.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-dd7136ad-daa6-46f9-b31a-1edd1c06ee80-0' usage_metadata={'input_tokens': 64, 'output_tokens': 21, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}}
Based on the provided text, breakfast is a meal that can include chocolate chip pancakes and scrambled eggs.
